# 0.1 Import packages and data 

In [ ]:
# Packages for data manipulation
import numpy as np
import pandas as pd
import json

In [ ]:
# Packages for dataviz
import plotly.express as px
import plotly.tools as tls
import cufflinks as cf
import matplotlib.pyplot as plt

In [ ]:
# configuration for plotly
template = "plotly_dark"

# offline configuration of cufflinks
cf.go_offline()

In [ ]:
# Load the data
data = pd.read_csv('../data/raw.csv')

In [ ]:
pd.set_option('display.max_columns', 110)

## Pieces of documentation about the variables
* https://rstudio-pubs-static.s3.amazonaws.com/365075_ec9ebe4da4cc465ba9beaef25cda6bad.html
* https://www.kaggle.com/dgomonov/new-york-city-airbnb-open-data

# 0.2 Quick look at the dataset

In [ ]:
data.head()

In [ ]:
pd.options.display.max_rows = 999

data.host_neighbourhood.value_counts()

In [ ]:
#data.columns
data.columns

In [ ]:
#data type
data.dtypes.value_counts()

To simplify this notebook, the cleanning of the variables is organized as followed:
* Quantitative
* Categorical 
* Boolean
* Text
* Spatial
* To drop
* I don't know yet

In [ ]:
# Outlier detection
#col_to_drop = []
row_to_drop = []

In [ ]:
# control objects
var_quant = []
var_cat = []
var_bool = []
var_text = []
var_date = []
var_space = []
var_to_drop = []
var_TODO = []
var_cible = []

# 1. Y variable

In [ ]:
var_cible.append("review_scores_rating")

On suppose qu'on ne dispose pas des autres *rating* pour expliquer Y

In [ ]:
var_to_drop.extend( ['review_scores_checkin', 'review_scores_cleanliness', 'review_scores_communication',
                     'review_scores_location', 'review_scores_accuracy', 'review_scores_value', 'reviews_per_month'] )

# 2. Data cleaning

# 2.1 Variables with no cleaning needed

There is no early remarks regarding theses variables. Nothing special came out when looking at them.

In [ ]:
var_quant.extend(["accommodates", "guests_included"])

In [ ]:
var_text.extend(['name', 'summary', 'space', 'description', 'neighborhood_overview', 'notes',
                'transit', 'access', 'interaction', 'house_rules', 'host_about'])

In [ ]:
var_space.extend(['latitude', 'longitude'])

Nous n'utilisons pas le zipcode dans notre traitement des variables categoriques

In [ ]:
var_to_drop.append("zipcode")

In [ ]:
data[var_text]

# 2.2 Variables to drop

## *id*

In [ ]:
data['id'].nunique()

In [ ]:
var_to_drop.append('id')

## 2.2.1 Empty columns

All these columns are filled with NAN so they are removed

In [ ]:
empty_l = ["thumbnail_url", "medium_url", "xl_picture_url",
                    "host_acceptance_rate", "neighbourhood_group_cleansed"]

In [ ]:
def print_empty(data, l):
    for col in l:
        print(data[col].isna().sum())
        print("")

In [ ]:
#print_empty(data, empty_l)

In [ ]:
var_to_drop.extend(empty_l)

In [ ]:
#print((data['experiences_offered'] == "none").sum())

In [ ]:
var_to_drop.append('experiences_offered')

## 2.2.2 Variables with one value

The variable is a constant so this won't be useful.

In [ ]:
one_l = ['state', 'country_code', 'country', 'has_availability',
            'jurisdiction_names','is_business_travel_ready', 'street']

In [ ]:
def print_unique(data, l):
    for col in l:
        print(data[col].value_counts())
        print("")

In [ ]:
#print_unique(data, one_l)

In [ ]:
var_to_drop.extend(one_l)

## 2.2.3 Useless variables

*scrape_id*, *calendar_last_scraped* and *last_scraped* are not useful in this study. We are not interested in the data scrapping.

In [ ]:
var_to_drop.extend(["scrape_id", "last_scraped", "calendar_last_scraped"])

The content of theses variables seems unclear and unecessary
* *minimum_minimum_nights*
* *maximum_minimum_nights*
* *minimum_maximum_nights*
* *maximum_maximum_nights*  

In [ ]:
var_to_drop.extend(['minimum_minimum_nights', 'maximum_minimum_nights', 
                    'minimum_maximum_nights', 'maximum_maximum_nights'])

Dans cette étude, nous allons aussi supprimer les variables contenant seulement des *url*.

In [ ]:
var_to_drop.extend(['listing_url', 'picture_url', 'host_url',
                    'host_thumbnail_url', 'host_picture_url'])

Le nom d'un hote ne parait pas pertinent dans cette étude.

In [ ]:
var_to_drop.extend(['host_name', 'host_id'])

In [ ]:
var_to_drop.append('calendar_updated')

La colonne Market contient 2 valeurs differentes. La valeur "D.C." n'a pas l'air abberante, nous allons donc garder cette ligne et enlever la colonne.

In [ ]:
data['market'].value_counts()

In [ ]:
var_to_drop.append('market')

In [ ]:
data[data['market'] == "D.C."]

La variable license n'a pas l'air exploitable. De plus, nous pensons qu'elle n'apporte pas grand chose à notre étude.

In [ ]:
var_to_drop.append('license')

On garde seulement la colonne availability_365

In [ ]:
data[['availability_30', 'availability_60', 'availability_90', 'availability_365']].head()

In [ ]:
var_quant.append('availability_365')
var_to_drop.extend(['availability_30', 'availability_60', 'availability_90'])

On garde juste la colonne number_of_reviews_ltm

In [ ]:
var_quant.append("number_of_reviews_ltm")
var_to_drop.append("number_of_reviews")

Entre *host_listings_count* et *calculated_host_listings_count*, nous conservons *host_listings_count* car celui ci contient plus d'informations. En effet, cette variable prend aussi en compte les logements des hotes non presents dans cette liste.

In [ ]:
var_to_drop.append('calculated_host_listings_count')

In [ ]:
tmp = data["host_listings_count"] - data["host_total_listings_count"]

In [ ]:
print(data["host_listings_count"].shape)
print(data["host_total_listings_count"].shape)
print(tmp.shape)
print(tmp.describe())

On constate que les colonnes *host_listings_count* et *host_total_listings_count*  sont identiques. On peut donc supprimer l'une des 2.  

In [ ]:
var_to_drop.append("host_total_listings_count")

On garde deja "host_listings_count", donc nous pensons que nous pouvons supprimer les variables suivantes, qui apportent presque la même information.

In [ ]:
var_to_drop.extend(['calculated_host_listings_count_entire_homes',
               'calculated_host_listings_count_private_rooms','calculated_host_listings_count_shared_rooms'])

In [ ]:
data['host_location'].value_counts()

Il y a trop de categories differents, nous choisissons de ne pas faire de regroupements donc nous supprimons ces variables

In [ ]:
var_to_drop.extend(['host_location', 'host_neighbourhood'])

# 2.3 Variables that need to be cast corectly

## 2.3.1 Simple cast

In [ ]:
float_to_int = ["minimum_nights_avg_ntm", "maximum_nights_avg_ntm"]

for col in float_to_int:
    data[col] = data[col].astype('int64')
    
var_quant.extend(float_to_int)

We cannot cast these values to int because of the NAN. It must remain floats.

In [ ]:
var_quant.extend(["bathrooms", "bedrooms", "beds", "host_listings_count"])

## 2.3.2 cast with missing values

In [ ]:
tot = data["square_feet"].shape
nb = data["square_feet"].isna().sum()

print("Nombre de valeurs ayant une superficie : "
      + str((tot - nb)[0]) )

print("Cela représente " + str(( (100*(tot - nb))/nb )[0]) + "% de nos données")

Il va falloir réflechir à comment traiter ces valeurs manquantes (dans le notebook suivant)

In [ ]:
var_to_drop.append("square_feet")

## 2.3.3 cast price to int

On convertit le prix en entier.

In [ ]:
price_to_int = ['price', 'extra_people', 'security_deposit', 'cleaning_fee']

In [ ]:
def money_to_int(x):
    if x != x:        # test si x == NAN
        return x
    else:
        return int( x[1 : (len(x)-4) ].replace(',', '') )

In [ ]:
for col in price_to_int:
    data[col] = data[col].map( lambda x: money_to_int(x) )

In [ ]:
var_quant.extend(price_to_int)

In [ ]:
def per_to_float(x):
    if x != x:        # test si x == NAN
        return x
    else:
        return  x[0 : -1 ]

In [ ]:
data['host_response_rate'] = data['host_response_rate'].map( lambda x: per_to_float(x) )

In [ ]:
var_quant.append('host_response_rate')

Est ce qu'on garde les autres prix? Je suis d'avis de les enlever

In [ ]:
var_to_drop.extend(['monthly_price', 'weekly_price'])

## 2.3.4 dates

In [ ]:
var_date.extend(['first_review', 'last_review', 'host_since'])

In [ ]:
for col in var_date:
    data[col] = pd.to_datetime(data[col])

In [ ]:
data[var_date].head()
#data[var_date].dtypes

In [ ]:
recent_date = data[var_date].max()

In [ ]:
for col in var_date:
    data[col] = recent_date[col] - data[col]
    data[col] = data[col].dt.days

# 2.4 Categorical variables

In [ ]:
data['city'].value_counts()

In [ ]:
data['smart_location'].value_counts()

In [ ]:
var_cat.append('city')
var_to_drop.append("smart_location")

Est ce qu'on a vraiment besoin des 2 variables ?  
On peut garder la variable *city* pour l'instant. On peut éventuellement retirer la ligne écrite en chinois.

In [ ]:
data['host_response_time'].value_counts()

In [ ]:
var_cat.append('host_response_time')

In [ ]:
cast_to_cat = ['property_type', 'room_type', 'bed_type', 'cancellation_policy']

In [ ]:
def print_cat(data, l):
    for col in l:
        print(data[col].value_counts())
        print("")

In [ ]:
#print_cat(data, cast_to_cat)

In [ ]:
var_cat.extend(cast_to_cat)

Il faudrait réduire le nombre de catégories de *property_type*.

## Colonnes *neighbourhood* et *neighbourhood_cleansed*

In [ ]:
data[['neighbourhood', 'neighbourhood_cleansed']].head()

La variable *neighbourhood_cleansed* a l'air d'être une version prétraitée de *neighbourhood*. NOus pouvons donc seulement garder *neighbourhood_cleansed*.  
Est ce qu'on traite cette colonne comme du texte ou comme une variable catégorique?

In [ ]:
var_space.append('neighbourhood_cleansed')

In [ ]:
var_to_drop.append('neighbourhood')

# 2.5 Boolean variables

In [ ]:
var_bool = ['requires_license', 'instant_bookable',
                'require_guest_profile_picture', 'require_guest_phone_verification',
                'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified']

In [ ]:
def print_bool(data, l):
    for col in l:
        print(data[col].value_counts())
        print("")

In [ ]:
#print_bool(data, var_bool)

In [ ]:
d = {'t': True, 'f': False}

In [ ]:
for col in var_bool:
    data[col] = data[col].map(d)

## Colonne *is_location_exact*

La variable is_location_exact ne semble pas être pertinente pour notre analyse. On la supprime.

In [ ]:
var_to_drop.append('is_location_exact')

## Colonnes *amenities* et *host_verifications*

Pour chaque logement, nous disosons d'une liste. Il y a trop de valeurs différentes pour en faire des categories. Nous allons donc faire un *count* de ces valeurs.

In [ ]:
data['amenities'].head()

In [ ]:
data['host_verifications'].head()

In [ ]:
amenities_count = []
host_verif_count = []

for row in data.itertuples(index=False):    
    amenities_count.append(len(row.amenities[1:-1].replace('"','').split(",")))
    host_verif_count.append(len(row.host_verifications[1:-1].replace('"','').split(",")))

In [ ]:
data['amenities'] = amenities_count
data['host_verifications'] = host_verif_count

In [ ]:
var_quant.extend(["amenities", 'host_verifications'])

# 2.6 Variables with outliers

## *minimum_nights* et *maximum_nights*

In [ ]:
def add_row_to_drop(data, variable, condition):
    l = []
    
    for value in condition:
        l.append(data[data[variable] == value].index[0])
    
    return l

In [ ]:
condition = [100000000, 1125, 1000]
row_to_drop.extend(add_row_to_drop(data, "minimum_nights", condition))

On constate des "paliers" pour les valeurs 30, 60, 90 et 365. Est ce qu'il y a un rapport avec les variables "availibility_xx". Si c'est le cas, on risque d'avoir un problème de multicolinéarité et il faudra surement faire une selection parmi ces variables.

In [ ]:
var_quant.append('host_verifications_count')

## 3.2 Hosts

In [ ]:
var_quant.extend(["maximum_nights", "minimum_nights"])

In [ ]:
#px.box(data, y="maximum_nights", points="all", template = template).show()

In [ ]:
#data_float.iplot(kind='box', filename='cufflinks/box-plots', world_readable=True,
#               columns = ['minimum_nights_avg_ntm', 'maximum_nights_avg_ntm'])

## *minimum_nights_avg_ntm* and *maximum_nights_avg_ntm*

D'après cette source (https://eprints.ucm.es/57354/1/TFM_Rentalbility_PriscillaToscano_VF_160919.pdf)  
minimum_nights_avg_ntm = Minimum Nights in Avg from last Twelve Month, ce qui me parait etrange

In [ ]:
condition = [100000000, 1125, 1000]
row_to_drop.extend(add_row_to_drop(data, "minimum_nights_avg_ntm", condition))

In [ ]:
condition = [2147483647, 100000, 10000, 9999, 2000, 1825]
row_to_drop.extend(add_row_to_drop(data, "maximum_nights_avg_ntm", condition))

## *city*

In [ ]:
host_close=pd.DataFrame(columns=['host_is_close'])

for row in data.itertuples(index = False): 
    host_close = host_close.append({'host_is_close': (row.host_location=='San Francisco, California, United States')}, ignore_index=True)

In [ ]:
#host_close

In [ ]:
data=data.join(host_close)

In [ ]:
var_to_drop.append('host_location')

In [ ]:
var_bool.append('host_is_close')

# 4. Check if all variables have been treated one time*

In [ ]:
control = var_quant + var_cat + var_bool + var_text + var_date + var_to_drop + var_TODO + var_cible + var_space

In [ ]:
print(len(control))
print(len(set(control)))
print(len(data.columns))

In [ ]:
len(var_cible + var_quant + var_cat + var_bool + var_space + var_date + var_text)

In [ ]:
len(var_to_drop)

# 5. Save the cleanned data

In [ ]:
row_to_drop.extend(data[data['review_scores_rating'].isna()].index)

In [ ]:
data_clean = data[var_cible + var_quant + var_cat + var_bool + var_space + var_date + var_text]
data_clean = data_clean.drop(row_to_drop)

In [ ]:
data_clean.head()

In [ ]:
row_to_drop[-1]

In [ ]:
meta_data = {}
meta_data['y'] = var_cible[0]
meta_data['quant'] = var_quant
meta_data['cat'] = var_cat
meta_data['bool'] = var_bool
meta_data['space'] = var_space
meta_data['date'] = var_date
meta_data['text'] = var_text

In [ ]:
with open('../data/meta_data.json', 'w') as outfile:
    json.dump(meta_data, outfile)
    
data_clean.to_csv ('../data/clean.csv',index=False, header=True)